In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import BitsAndBytesConfig

model_path = "google/mt5-small"
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained(
    model_path,
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_path,
)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/

In [4]:
from datasets import DatasetDict
from datasets import load_dataset

dataset = load_dataset("IlyaGusev/gazeta")

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1491: FutureWarning: The repository for IlyaGusev/gazeta contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/gazeta
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [6]:
train_subset = dataset['train'].select(range(6000))
test_subset = dataset['test'].select(range(600))
validation_subset = dataset['validation'].select(range(600))

# Combine them back into a DatasetDict if needed
subdataset = DatasetDict({
    'train': train_subset,
    'test': test_subset,
    'validation': validation_subset
})

In [7]:
def tokenize_fn(example):
    model_inputs = tokenizer(example['text'], padding=True, truncation=True, max_length=512)
    model_inputs["labels"] = tokenizer(example['summary'], padding=True, truncation=True, max_length=512)['input_ids']
    return model_inputs

inputs = subdataset.map(tokenize_fn, batched = True)
inputs = inputs.map(remove_columns=['title', 'date', 'url'])

inputs.set_format(type='torch')

In [9]:
from transformers import Trainer
from transformers import DataCollatorWithPadding, DataCollatorForSeq2Seq

2024-06-08 18:52:42.383514: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-08 18:52:42.383568: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-08 18:52:42.384939: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, 
    model=model
)

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="test-trainer", 
    num_train_epochs = 5,
    eval_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100, 
    per_device_train_batch_size = 10,
    per_device_eval_batch_size = 10,
    report_to = 'none',
)

In [13]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=inputs["train"],
    eval_dataset=inputs["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [14]:
trainer.train()


/opt/conda/lib/python3.10/site-packages/transformers/data/data_collator.py:646: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:261.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss
100,28.619500,13.089636
200,16.193800,8.211783
300,11.332000,6.855174
400,8.504800,6.262938
500,6.375500,5.398847
600,4.911500,4.550810
700,3.953600,3.081573
800,3.464100,2.537444
900,3.080800,2.275347
1000,2.941200,2.003653


RuntimeError: [enforce fail at inline_container.cc:424] . unexpected pos 1350317568 vs 1350317460

In [15]:
from transformers import pipeline

In [26]:
pipe = pipeline("summarization", 
                model=model, 
                tokenizer=tokenizer, 
                max_length = 40, 
               )

In [31]:
my_inputs = "Один раз школьник Ваня отправился в поход в горы с классом. Они выехали рано утром, и дорога заняла несколько часов. Когда автобус остановился, перед ними открылась величественная картина горных вершин. Ребята сразу же начали подниматься по тропинке, наслаждаясь свежим воздухом и красивыми видами. Ваня был в восторге от природы вокруг: цветы, деревья и маленькие ручейки создавали сказочную атмосферОни шли долго, иногда останавливаясь на привалы, чтобы отдохнуть и перекусить. Учитель рассказывал интересные факты о горах, и Ваня узнал много нового. В какой-то момент они добрались до вершины, откуда открывался потрясающий вид на долину. Ваня чувствовал себя на вершине мира и радовался, что смог покорить эту высоту. На обратном пути все делились впечатлениями и мечтали о новых походах. Этот день навсегда остался в памяти Вани как один из самых ярких и увлекательных."

In [33]:
pipe(my_inputs)

[{'summary_text': 'В ходе школьника Ваня отправился в поход в горы с классом.'}]

## Итог

На Данный момент работает не слишком хорошо, однако обучение длилось всего 30 минут, поэтому результат приемлимый.